In [18]:
import pathlib
import fnmatch
import shutil

In [19]:
def find_project_root(starting_path: pathlib.Path = pathlib.Path.cwd(), marker: str = ".git") -> pathlib.Path:
    """
    Sube desde el directorio 'starting_path' hasta encontrar un directorio que contenga
    el marcador (por defecto, ".git"), considerándolo la raíz del proyecto.
    Si no se encuentra, retorna 'starting_path'.
    """
    current = starting_path.resolve()
    for parent in [current] + list(current.parents):
        if (parent / marker).exists():
            return parent
    return current

def find_candidates(root: pathlib.Path, deletion_patterns: list) -> list:
    """
    Recorre recursivamente el directorio 'root' y devuelve una lista de Path que
    coinciden con alguno de los patrones en 'deletion_patterns'.
    """
    candidates = []
    for item in root.rglob('*'):
        for pattern in deletion_patterns:
            if fnmatch.fnmatch(item.name, pattern):
                candidates.append(item)
                break
    return candidates

def filter_exclusions(candidates: list, exclusion_patterns: list) -> list:
    """
    De la lista de candidatos, filtra aquellos que coincidan con alguno de los patrones
    en 'exclusion_patterns'. Se comprueba tanto el nombre del elemento como los nombres
    de sus carpetas padres. Así, si se excluye el nombre de una carpeta, se omite todo su contenido.
    """
    filtered = []
    for item in candidates:
        exclude_item = False
        for pattern in exclusion_patterns:
            # Comprobar si el nombre del elemento coincide con el patrón.
            if fnmatch.fnmatch(item.name, pattern):
                exclude_item = True
                break
            # Comprobar si alguno de los padres coincide.
            for parent in item.parents:
                if fnmatch.fnmatch(parent.name, pattern):
                    exclude_item = True
                    break
            if exclude_item:
                break
        if not exclude_item:
            filtered.append(item)
    return filtered

def delete_candidates(candidates: list):
    """
    Elimina cada uno de los elementos de la lista 'candidates'. Para archivos o enlaces se usa unlink()
    y para directorios se emplea shutil.rmtree.
    """
    for item in candidates:
        try:
            if item.is_file() or item.is_symlink():
                print(f"Eliminando archivo: {item}")
                item.unlink()
            elif item.is_dir():
                print(f"Eliminando carpeta: {item}")
                shutil.rmtree(item)
        except Exception as e:
            print(f"Error al eliminar {item}: {e}")

def clean_project(root: pathlib.Path, deletion_patterns: list, exclusion_patterns: list):
    """
    Función principal:
      1. Recorre recursivamente 'root' buscando elementos que coincidan con 'deletion_patterns'.
      2. Lista todos los elementos encontrados.
      3. Aplica los patrones de exclusión (se verifica el nombre del elemento y de sus padres).
      4. Muestra la lista final y pide confirmación para proceder a eliminar los elementos.

    Parámetros:
      - root: Directorio raíz desde donde comenzar la búsqueda.
      - deletion_patterns: Lista de patrones de nombres (pueden incluir comodines, e.j., "*.parquet").
      - exclusion_patterns: Lista de patrones a excluir; si se especifica el nombre de una carpeta,
        se excluye esa carpeta y todo su contenido.
    """
    print(f"Buscando elementos en {root} que coincidan con patrones de eliminación...")
    candidates = find_candidates(root, deletion_patterns)
    print(f"Se encontraron {len(candidates)} elementos:")
    for item in candidates:
        print(f"  - {item}")

    if exclusion_patterns:
        print("\nAplicando patrones de exclusión...")
        final_candidates = filter_exclusions(candidates, exclusion_patterns)
    else:
        final_candidates = candidates

    print(f"\nSe eliminarán {len(final_candidates)} elementos tras aplicar exclusiones:")
    for item in final_candidates:
        print(f"  - {item}")

    confirm = input("\n¿Deseas proceder con la eliminación? (s/n): ")
    if confirm.lower() == 's':
        delete_candidates(final_candidates)
        print("Proceso de limpieza completado.")
    else:
        print("Proceso de limpieza cancelado.")

In [20]:
project_root = find_project_root(marker=".git")
print(f"Raíz del proyecto detectada: {project_root}")

Raíz del proyecto detectada: D:\Espacios de trabajo\personal-library


In [21]:
# Lista de patrones de eliminación (ejemplo: archivos y carpetas que se desean borrar)
deletion_patterns = ["__pycache__", "*.parquet"]

# Lista de exclusión: si se especifica el nombre de una carpeta, se excluye esa carpeta y todo su contenido.
exclusion_patterns = [".venv"]

In [22]:
# Ejecuta el proceso de limpieza a partir de la raíz del proyecto.
clean_project(project_root, deletion_patterns, exclusion_patterns)

Buscando elementos en D:\Espacios de trabajo\personal-library que coincidan con patrones de eliminación...
Se encontraron 3 elementos:
  - D:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\__pycache__
  - D:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\win32\lib\__pycache__
  - D:\Espacios de trabajo\personal-library\.venv\Lib\site-packages\_distutils_hack\__pycache__

Aplicando patrones de exclusión...

Se eliminarán 0 elementos tras aplicar exclusiones:
Proceso de limpieza completado.
